In [2]:
import numpy as np
import hickle as hkl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio as rs
import os
import sys
import boto3
import yaml

sys.path.append('../src/')
from utils import quick_viz as make
%load_ext autoreload
%autoreload 2

## Conclusions
- A total of 255 tiles are affected by haze.
- In areas where resegmentation did not help the predictions, we expect the features to play a role. This notebook visualizes 65 features for neighboring haze affected tiles where resegmentation appeared successful (pd_east) and unsuccessful (pd_west).
- Artifacts are most extreme for the following feature indices pd_east: `10, 17, 19, 20, 31` and pd_west: `17, 19, 20, 23, 25, 28, 29, 31, 32, 40`. 
- The top 40 features used in the current model includes feature `42` which had strong artifacts in pd_west

In [38]:
east = [10, 17, 19, 20, 31]
west = [17, 19, 20, 23, 25, 28, 29, 31, 32, 40]
east = [i+13 for i in east]
west = [i+13 for i in west]
east, west

([23, 30, 32, 33, 44], [30, 32, 33, 36, 38, 41, 42, 44, 45, 53])

top features for v25.5 model (current): `0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 18, 20, 21, 24, 34, 42, 50, 52, 59, 60, 62, 63, 64, 65, 70, 71, 72, 73, 74, 76, 77, 84, 89, 90, 93`

## Example: resegmentation is successful

### ARD

In [29]:
# tile_idx_a = (1656, 1068)
# tile_idx_b = (1657, 1068)
# ard_a = hkl.load(f'../tmp/ghana/{str(tile_idx_a[0])}/{str(tile_idx_a[1])}/ard/{str(tile_idx_a[0])}X{str(tile_idx_a[1])}Y_ard.hkl')
# ard_b = hkl.load(f'../tmp/ghana/{str(tile_idx_b[0])}/{str(tile_idx_b[1])}/ard/{str(tile_idx_b[0])}X{str(tile_idx_b[1])}Y_ard.hkl')
# for i in range(0, 13):
#     make.heat_combine_neighbors(ard_a, ard_b, i, f"Left:{tile_idx_a}  Right:{tile_idx_b}, Index: {i}")

In [31]:
tile_idx_a = (1656, 1068)
tile_idx_b = (1657, 1068)
feats_a = hkl.load(f'../tmp/ghana/{str(tile_idx_a[0])}/{str(tile_idx_a[1])}/raw/feats/{str(tile_idx_a[0])}X{str(tile_idx_a[1])}Y_feats.hkl')
feats_b = hkl.load(f'../tmp/ghana/{str(tile_idx_b[0])}/{str(tile_idx_b[1])}/raw/feats/{str(tile_idx_b[0])}X{str(tile_idx_b[1])}Y_feats.hkl')
feats_a = np.rollaxis(feats_a, 0, 3)
feats_a = np.rollaxis(feats_a, 0, 2)
feats_b = np.rollaxis(feats_b, 0, 3)
feats_b = np.rollaxis(feats_b, 0, 2)
feats_a.shape, feats_b.shape

((618, 614, 65), (618, 614, 65))

In [39]:
# for i in range(0, 65):
#     make.heat_combine_neighbors(feats_a, feats_b, i, f"Left:{tile_idx_a}  Right:{tile_idx_b}, Index: {i}")

## Example: resegmentation is unsuccessful

### ARD

In [30]:
tile_idx_a = (1647, 1079)
tile_idx_b = (1648, 1079)
ard_a = hkl.load(f'../tmp/ghana/{str(tile_idx_a[0])}/{str(tile_idx_a[1])}/ard/{str(tile_idx_a[0])}X{str(tile_idx_a[1])}Y_ard.hkl')
ard_b = hkl.load(f'../tmp/ghana/{str(tile_idx_b[0])}/{str(tile_idx_b[1])}/ard/{str(tile_idx_b[0])}X{str(tile_idx_b[1])}Y_ard.hkl')
for i in range(0, 13):
    make.heat_combine_neighbors(ard_a, ard_b, i, f"Left:{tile_idx_a}  Right:{tile_idx_b}, Index: {i}")

### Features

In [34]:
tile_idx_a = (1647, 1079)
tile_idx_b = (1648, 1079)
feats_a = hkl.load(f'../tmp/ghana/{str(tile_idx_a[0])}/{str(tile_idx_a[1])}/raw/feats/{str(tile_idx_a[0])}X{str(tile_idx_a[1])}Y_feats.hkl')
feats_b = hkl.load(f'../tmp/ghana/{str(tile_idx_b[0])}/{str(tile_idx_b[1])}/raw/feats/{str(tile_idx_b[0])}X{str(tile_idx_b[1])}Y_feats.hkl')
feats_a = np.rollaxis(feats_a, 0, 3)
feats_a = np.rollaxis(feats_a, 0, 2)
feats_b = np.rollaxis(feats_b, 0, 3)
feats_b = np.rollaxis(feats_b, 0, 2)
feats_a.shape, feats_b.shape

((618, 614, 65), (618, 614, 65))

In [40]:
# for i in range(0, 65):
#     make.heat_combine_neighbors(feats_a, feats_b, i, f"Left:{tile_idx_a}  Right:{tile_idx_b}, Index: {i}")